<a href="https://colab.research.google.com/github/kirinsannnnnnnnnn/image_search_hello_world/blob/master/hello_ramen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
is_colab = 'GCS_READ_CACHE_BLOCK_SIZE_MB' in os.environ

In [2]:
## for colab
if is_colab:
    from google.colab import drive
    drive.mount('./gdrive')
    ! pip install annoy
    ! mkdir -p gdrive/My\ Drive/raw_data gdrive/My\ Drive/intermediate_data
    ! cd gdrive/My\ Drive/raw_data
    ! wget https://github.com/K-jun1221/reverse-image-search/raw/master/ramen_images.zip
    ! unzip ramen_images.zip
    ! rm -rf __MACOSX ramen_images.zip

Mounted at ./gdrive
     |████████████████████████████████| 645kB 4.8MB/s 
  Created wheel for annoy: filename=annoy-1.16.2-cp36-cp36m-linux_x86_64.whl size=310425 sha256=b5d143d39b719d7701ceeab5ce9cb953ad3b3ad6e171d31ba172a31bf469c71e
  Stored in directory: /root/.cache/pip/wheels/d3/d7/68/3795670ef4c6781fc10df0d6cf83b922244aa28cd9489d1176
Successfully built annoy
--2019-11-18 15:37:52--  https://github.com/K-jun1221/reverse-image-search/raw/master/ramen_images.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/K-jun1221/reverse-image-search/master/ramen_images.zip [following]
--2019-11-18 15:37:52--  https://raw.githubusercontent.com/K-jun1221/reverse-image-search/master/ramen_images.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to r

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import cv2
import os

from IPython.display import Image
from pathlib import Path

from keras.applications.vgg19 import VGG19
from keras_preprocessing import image
from keras.models import Model

from annoy import AnnoyIndex
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input

Using TensorFlow backend.


ModuleNotFoundError: ignored

In [0]:
## for local docker image
! ls ./raw_data/ramen_images/ | head

In [0]:
data_dir = Path('raw_data/ramen_images/')
if is_colab:
    data_dir = Path('gdrive/My\ Drive') / data_dir
intermediate_dir = Path("intermediate_data")

img_path = data_dir / Path("ramen1.jpg")
Image(img_path)

In [0]:
base_model = VGG19(weights="imagenet")
base_model.summary()

In [0]:
print(base_model.input)
print(base_model.output)

In [0]:
img = image.load_img(img_path, target_size=(224, 224))
input = image.img_to_array(img)
result = base_model.predict(np.array([input]))
print("配列の中身", result)
print("配列の長さ: ", len(result[0]))

In [0]:
type(img), type(result), result.shape

In [0]:
model = Model(
  inputs=base_model.input,
  outputs=base_model.get_layer("fc2").output
)

In [0]:
input = image.img_to_array(img)
result = model.predict(np.array([input]))
print("実際の値", result)
print("配列の長さ: ", len(result[0]))

In [0]:
%%time

dim = 4096 
annoy_model = AnnoyIndex(dim)

for i in range (1, 11):
    img_path = data_dir / "ramen" +str(i)+ ".jpg"
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    fc2_features = model.predict(x)
    annoy_model.add_item(i, fc2_features[0])

In [0]:
annoy_model.build(10)
annoy_model.save(intermediate_dir / "ramen_images_next.ann")

In [0]:
items = trained_model.get_nns_by_item(2, 3, search_k=-1, include_distances=False)
print(items)